**Step 1: Find all patients with condition X from three sources**

**Required**
* Connect to FHIR Synthetic Patient Data
* Connect to OHDSI Synthetic Patient Data
* Connect to HUSH+ Synthetic Patient Data

**Step 1.1: Find 

In [20]:
## All the imports we need
from urllib2 import Request, urlopen, URLError
from urllib import quote_plus

import mysql.connector

import pprint, json, requests

from datetime import datetime, timedelta
from dateutil.parser import parse as parse_date
from greentranslator.api import GreenTranslator

DISEASE2SYMPTOMS = [x.split("\t") for x in urlopen("https://www.nature.com/article-assets/npg/ncomms/2014/140626/ncomms5212/extref/ncomms5212-s4.txt").read().split("\n")]

In [6]:
#connect to UMLS
try:
  cnx = mysql.connector.connect(user='tadmin',
                                password='ncats_translator!',
                                database='umls',
                                host='translator.ceyknq0yekb3.us-east-1.rds.amazonaws.com')
except mysql.connector.Error as err:
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Something is wrong with your user name or password")
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  else:
    print(err)
else:
  print
  #cnx.close()

In [7]:
#Define functions
def findMeshAndICD10(name):
    cursor = cnx.cursor()
    query = ("SELECT CUI FROM umls.MRCONSO WHERE STR='"+name+"'")
    cursor.execute(query, ())
    res = "Undef"
    for code in cursor:
        if res=="Undef":
            res = code
    if res != "Undef":
        query = ("SELECT CODE FROM umls.MRCONSO WHERE SAB='MSH' AND CUI='"+res[0]+"'")
        cursor.execute(query, ())
        msh = "Undef"
        for code in cursor:
            msh = code
        query = ("SELECT CODE FROM umls.MRCONSO WHERE SAB='ICD10' AND CUI='"+res[0]+"'")
        cursor.execute(query, ())
        icd10 = "Undef"
        for code in cursor:
            icd10 = code
        query = ("SELECT CODE FROM umls.MRCONSO WHERE SAB='ICD10CM' AND CUI='"+res[0]+"'")
        cursor.execute(query, ())
        for code in cursor:
            icd10 = code
        return (msh[0],icd10[0])
    return ("Undef","Undef") 


In [8]:
findMeshAndICD10('Asthma')

(u'D001249', u'J45.90')

In [9]:
## FHIR patient data

base_url = 'http://ictrweb.johnshopkins.edu/rest/synthetic'
patients = base_url+"/Patient"+"?_count=50"

req = requests.get(patients)

In [12]:
## Given disease/condition term, get back ICD codes
def txt2icd(txt):
    url_con = "http://api.ohdsi.org/WebAPI/vocabulary/search"
    headers = {'content-type': 'application/json'}
    params = {"QUERY": "Asthma",
              "VOCABULARY_ID": ["ICD9CM"]
    }
    response = requests.post(url_con, data=json.dumps(params), headers=headers)
    data= json.loads(response.text.decode('utf-8'))
    icd9arr=[]
    for d in data:
        icd9arr.append(d["CONCEPT_CODE"])
    return icd9arr
print txt2icd('asthma')

[u'E945.7', u'493', u'493.9', u'493.90', u'493.92', u'493.91', u'493.2', u'493.20', u'493.22', u'493.21', u'493.82', u'493.0', u'493.00', u'493.02', u'493.01', u'V17.5', u'493.1', u'493.10', u'493.12', u'493.11', u'493.8', u'975.7']


In [29]:
## Given disease name, get back symptoms (defined using MeSH terms) along with TFIDF scores
def disease2symptom(txt):
    print DISEASE2SYMPTOMS[:5]
    s = filter(lambda x: x[1].lower(), DISEASE2SYMPTOMS[:5])
    print s
    return(s)
print disease2symptom("Asthma")

[['MeSH Symptom Term', 'MeSH Disease Term', 'PubMed occurrence', 'TFIDF score'], ['Aging, Premature', 'Respiratory Syncytial Virus Infections', '1', '3.4645514895379677'], ['Aging, Premature', 'Orthomyxoviridae Infections', '1', '3.4645514895379677'], ['Aging, Premature', 'HIV Infections', '3', '10.393654468613903'], ['Aging, Premature', 'Acquired Immunodeficiency Syndrome', '3', '10.393654468613903']]
[['MeSH Symptom Term', 'MeSH Disease Term', 'PubMed occurrence', 'TFIDF score'], ['Aging, Premature', 'Respiratory Syncytial Virus Infections', '1', '3.4645514895379677'], ['Aging, Premature', 'Orthomyxoviridae Infections', '1', '3.4645514895379677'], ['Aging, Premature', 'HIV Infections', '3', '10.393654468613903'], ['Aging, Premature', 'Acquired Immunodeficiency Syndrome', '3', '10.393654468613903']]


NameError: global name 'symptoms' is not defined

In [23]:
## Pull patients matching a list of ICD codes
query = GreenTranslator ().get_query ()

patients = query.clinical_get_patients (age='8', sex='male', 
                                        race='white', location='OUTPATIENT')
##pprint.pprint (patients)

Initializing thread provenance @thread: MainThread


In [9]:
meds = {}
for x in patients:
    medList = x['medList']
    # Collect the unique meds
    for m in medList.keys():
        found = False
        try:
            meds[medList[m]] = meds[medList[m]]+1
        except KeyError:
            meds[medList[m]] = 1
pprint.pprint(meds)

{None: 9,
 u'0.3 ML Epinephrine 1 MG/ML Auto-Injector [Epipen]': 1,
 u'120 ACTUAT Fluticasone propionate 0.05 MG/ACTUAT Nasal Inhaler': 2,
 u'24 HR Methylphenidate Hydrochloride 10 MG Extended Release Oral Capsule [Ritalin]': 1,
 u'ACYCLOVIR 5 % TOPICAL OINTMENT': 1,
 u'ADVAIR HFA 230-21 MCG INHALER': 1,
 u'ALBUTEROL 90 MCG INHALER': 1,
 u'ALBUTEROL SULF HFA 90 MCG INH': 7,
 u'ALBUTEROL SULFATE 2.5 MG/3 ML (0.083 %) SOLUTION FOR NEBULIZATION': 9,
 u'ALBUTEROL SULFATE HFA 90 MCG/ACTUATION AEROSOL INHALER': 18,
 u'AMOXICILLIN 250 MG CHEWABLE TABLET': 1,
 u'AMOXICILLIN 875 MG TABLET': 1,
 u'AUGMENTIN 400-57 MG/5 ML': 1,
 u'AUGMENTIN ES-600 SUSPENSION': 3,
 u'AZITHROMYCIN 250 MG TABLET': 11,
 u'Acetaminophen 21.7 MG/ML / Hydrocodone Bitartrate 0.5 MG/ML Oral Solution': 1,
 u'Acetaminophen 32 MG/ML Oral Suspension': 1,
 u'Albuterol 0.83 MG/ML Inhalant Solution': 3,
 u'Amitriptyline Hydrochloride 10 MG Oral Tablet': 1,
 u'Amoxicillin 120 MG/ML / Clavulanate 8.58 MG/ML Oral Suspension': 4,
 u

In [79]:
## For a given medication string get NCBO annotations
## We let NCBO match any ontology since just using RxNORM doesn't
## always give us just the drug name (e.g., "CLINDAMYCIN 15 MG/ML ORAL SOLUTION" is
## a valid RxNORM term)
def med2rxnorm(txt):
    url = 'http://data.bioontology.org/annotator?text=%s&apikey=b792dd1b-cdc2-4cc8-aaf2-4fa4fbf47e4e'
    txt = urlopen(url % quote_plus(txt)).read()
    resp = json.loads(txt)
    if len(resp) == 0: return([])
    annos = []
    for aresp in resp:
        annos.extend([ x['text'] for x in aresp['annotations'] ])
    ## filter for terms that have no numbers as a hack to just the drug name
    annos = filter(lambda x: not any(d in x for d in'0123456789'), annos)
    return(annos)
print med2rxnorm("CLINDAMYCIN 15 MG/ML ORAL SOLUTION")        


[u'CLINDAMYCIN', u'ORAL SOLUTION']


In [80]:
## Get RxNORM codes for medication strings
medrxnorm = {}
for med in meds.keys():
    if med is None: continue    
    annos = med2rxnorm(med)
    print 'Processing %s and found %d annotations' % (med, len(annos))
    medrxnorm[med] = {'count':meds[med], 'annos':annos}    

Processing HYDROCORTISONE VAL 0.2% CREAM and found 1 annotations
Processing CLARITIN 5 MG/5 ML SYRUP and found 1 annotations
Processing CLINDAMYCIN 150 MG/ML INJECTION SOLUTION and found 2 annotations
Processing PROAIR HFA 90 MCG INHALER and found 1 annotations
Processing ALBUTEROL SULFATE 2.5 MG/3 ML (0.083 %) SOLUTION FOR NEBULIZATION and found 2 annotations
Processing Fexofenadine hydrochloride 30 MG Oral Tablet and found 3 annotations
Processing ADVAIR HFA 230-21 MCG INHALER and found 1 annotations
Processing Ofloxacin 3 MG/ML Ophthalmic Solution and found 2 annotations
Processing levocetirizine dihydrochloride 5 MG Oral Tablet and found 3 annotations
Processing DIAZEPAM 2 MG TABLET and found 1 annotations
Processing Clonidine Hydrochloride 0.1 MG Oral Tablet and found 3 annotations
Processing LIDEX 0.05% OINTMENT and found 0 annotations
Processing KETOCONAZOLE 2 % TOPICAL CREAM and found 2 annotations
Processing BACTROBAN 2% OINTMENT and found 1 annotations
Processing FOCALIN XR 5

Processing Triamcinolone Acetonide 0.001 MG/MG Topical Ointment and found 3 annotations
Processing Glucose 50 MG/ML / Sodium Chloride 0.154 MEQ/ML Injectable Solution and found 4 annotations
Processing SUPRAX 200 MG/5 ML SUSPENSION and found 1 annotations
Processing LORTAB 7.5-500 MG/15 ML ELIXIR and found 1 annotations
Processing lisdexamfetamine dimesylate 50 MG Oral Capsule and found 3 annotations
Processing lansoprazole 30 MG Delayed Release Oral Capsule and found 3 annotations
Processing TAMIFLU 12 MG/ML SUSPENSION and found 1 annotations
Processing Amoxicillin 500 MG Oral Capsule and found 2 annotations
Processing Azithromycin 40 MG/ML Oral Suspension and found 2 annotations
Processing CONCERTA ER 27 MG TABLET and found 1 annotations
Processing Cyproheptadine hydrochloride 4 MG Oral Tablet and found 3 annotations
Processing AMOXICILLIN 250 MG CHEWABLE TABLET and found 2 annotations
Processing moxifloxacin 5 MG/ML Ophthalmic Solution and found 2 annotations
Processing AUGMENTIN ES